### This file has the implementation of  machine learning models on cleaned data.

### Importing Necessary Library

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
file_path = "/content/drive/My Drive/clean_data_capstoneV4.csv"
df = pd.read_csv(file_path,encoding='utf8', engine='python')
df.head()
#df=pd.read_csv("clean_data_capstoneV1.csv")

,clean_Short_desc,clean_Description,Assignment group
0,login issue,verified detail employee manager check ad rese...,GRP_0
1,log to vpn,receive log to vpn best,GRP_0
2,unable to access hr_tool page,unable to access hr_tool page,GRP_0
3,unable to log to engineering tool skype,unable to log to engineering tool skype,GRP_0
4,event critical hostname_221 com value mountpoi...,event critical hostname_221 com value mountpoi...,GRP_1


In [4]:
df.isnull().sum()

clean_Short_desc     0
clean_Description    0
Assignment group     0
dtype: int64

In [5]:
df['MergedColumn'] = df[df.columns[0:2]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)

data_df = df.drop(['clean_Short_desc','clean_Description'],axis=1)
data_df.head()

,Assignment group,MergedColumn
0,GRP_0,login issue verified detail employee manager c...
1,GRP_0,log to vpn receive log to vpn best
2,GRP_0,unable to access hr_tool page unable to access...
3,GRP_0,unable to log to engineering tool skype unable...
4,GRP_1,event critical hostname_221 com value mountpoi...


### Naive Bayes


In [6]:
### Count Vectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(df['MergedColumn']).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,df['Assignment group'], test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



(4559, 2500)
(4559,)
(1140, 2500)
(1140,)


In [8]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)

predictions = nb.predict(X_test)

print('Training accuracy',nb.score(X_train,y_train))

from sklearn.metrics import accuracy_score
print("Testing accuracy " ,accuracy_score(predictions,y_test))

Training accuracy 0.7729765299407765
Testing accuracy  0.6271929824561403


### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
print("Training accuracy",lr.score(X_train,y_train))

predictions = lr.predict(X_test)

from sklearn.metrics import accuracy_score
print("Testing accuracy",accuracy_score(predictions,y_test))

Training accuracy 0.9370475981574907
Testing accuracy 0.6657894736842105


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Overfitting as training accuracy is high and testing is low

### K Nearest Neighbours


In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5,weights='distance')
knn.fit(X_train,y_train)
print("Training accuracy",knn.score(X_train,y_train))

predictions = knn.predict(X_test)

from sklearn.metrics import accuracy_score
print("Testing accuracy",accuracy_score(predictions,y_test))

Training accuracy 0.9640271989471375
Testing accuracy 0.6149122807017544


Overfitting as training accuracy is high and testing is low

### SVM

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

encoder = LabelEncoder()
train_y = encoder.fit_transform(y_train)
test_y = encoder.fit_transform(y_test)

tfidfVect = TfidfVectorizer(max_features=5000)
X = tfidfVect.fit_transform(df['MergedColumn']).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,df['Assignment group'], test_size=0.2, random_state=42)


In [13]:
from sklearn.svm import SVC
svc= SVC(C=1.0,kernel='linear',degree=3,gamma='auto')
svc.fit(X_train,y_train)

print("Training accuracy",svc.score(X_train,y_train))

predictions = svc.predict(X_test)

from sklearn.metrics import accuracy_score
print("Testing accuracy",accuracy_score(predictions,y_test))

Training accuracy 0.823206843606054
Testing accuracy 0.6587719298245615


### Ckecking by removing group 0

In [14]:
data_df1 = data_df[data_df['Assignment group'] != 'GRP_0']
data_df1.head()

,Assignment group,MergedColumn
4,GRP_1,event critical hostname_221 com value mountpoi...
14,GRP_3,undocking pc screen come back undocking pc scr...
22,GRP_4,duplication network address receive gentle two...
30,GRP_5,reroute job printer01 to printer02 issue to re...
34,GRP_6,job job_1424 fail job_scheduler receive job jo...


In [16]:
### Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(data_df1['MergedColumn']).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,data_df1['Assignment group'], test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)

predictions = nb.predict(X_test)

print('Training accuracy',nb.score(X_train,y_train))

from sklearn.metrics import accuracy_score
print("Testing accuracy " ,accuracy_score(predictions,y_test))

(2578, 2500)
(2578,)
(645, 2500)
(645,)
Training accuracy 0.7498060512024826
Testing accuracy  0.5333333333333333
